In [2]:
def dir_view(path='.'):
    import os
    for item in os.listdir(path):
        if os.path.isdir(item):
            print('D ' + item)
        elif os.path.isfile(item):
            print('F ' + item)
        else:
            print('? ' + item)

def get_ext(item):
    sn = item.split('.')
    if len(sn) > 1:
        return sn[-1]
    #TODO: return item.rsplit('.', 1)[-1]
    
def mult_cond(string, cond, separator=' '):
    if cond in '':
        return True
    for i in cond.split(separator):
        if string.find(i) > -1:
            return False
    return True

def get_type(item):
    from os import path
    if path.isfile(item):
        return 'F'
    elif path.isdir(item):
        return 'D'
    else:
        return '?'
            
def get_all_files(path='.', only=''):
    import os
    all_files = []
    for item in os.listdir(path):
        if os.path.isfile(item):
            if only not in '' and get_ext(item) in only.split(' '):
                all_files.append(item)
            elif only in '':
                all_files.append(item)
    return all_files

def convert_to_mp4(items):
    job_list = []
    for item in items:
        job_list.append('ffmpeg -i %s %s' % (item, item.replace('avi', 'mp4')))
    return job_list

def save_bat(jobs):
    f = open('convert.bat', 'w+')
    for job in jobs:
        f.write(job+'\n')
    f.write('del *.avi\n')
    f.write('del *.bat\n')
    f.close()
    
def create_bat():
    ask_to_save = input('Create job file? [y/N]')
    if ask_to_save.lower() in ('n', '', ' '):
        print('Job NOT created.')
    else:
        save_bat(jobs)
        print('Job created. Run this file manualy.')

class Selector:
    
    def __init__(self, path='.', goto=False):
        self.selected = []
        self.path = path
        if goto:
            from os import chdir
            chdir(self.path)
            self.path = '.'
    
    def select(self, ins='', dis='', typ='F'):
        from os import listdir
        for item in listdir(self.path):
            if not mult_cond(get_type(item), typ):
                if ins not in '' and mult_cond(item, dis) and get_ext(item) in ins.split(' '):
                    self.selected.append(item)
                if ins in ('', '*', ' '):
                    self.selected.append(item)
    def info(self):
        print('T: %d' % (len(self.selected)), end='\n----------\n')
        for i,j in enumerate(self.selected):
            print('%d %s' % (i+1, j))
    
    def get(self):
        return self.selected

In [3]:
s = Selector(path=r'E:\\', goto=True)
s.select(typ='D')
s.info()

T: 13
----------
1 $RECYCLE.BIN
2 3D
3 backups
4 Books
5 GameSaves
6 images
7 Jul_backup
8 projects
9 System Volume Information
10 tmp
11 torrents
12 video
13 youtube


In [4]:
s = Selector()
s.select(ins='pdf djvu')
s.info()

T: 0
----------


In [6]:
s = Selector(path=r"E:\torrents\Uzhnyj.park.(01-13.sezon).1997-2009.XviD."
             r"DVDRip\Uzhnyj.park.(05.sezon.01-14.serii).2001.XviD.DVDRip", goto=True)
s.select(ins='avi')
s.info()
jobs = convert_to_mp4(s.selected)
create_bat()

T: 0
----------
Create job file? [y/N]
Job NOT created.


In [16]:
#s.select('type=D include=pro case=true')
import os
path = r'e:\torrents'
s = Selector(path)
s.select()

ss = s.get()
# print(len(ss))

def get_args_dict(string):
    arglist = string.split(' ')
    argdict = dict()
    for arg in arglist:
        if arg.find('=') > -1:
            a, b = arg.split('=')
            if a not in argdict:
                argdict[a] = b
            else:
                argdict[a] += ',' + b
    return argdict

#sring = 'type=dir include=pro case=true include=one,two,three four bad=par, par2 bad2=par '
#get_args_dict(string)

query = 'text=intel case=false'

def search(db, query):
    qdict = get_args_dict(query)
    for item in db:
        if not mult_cond(item, qdict['text'], separator=','):
            print(os.path.getatime(item))
            
search(ss, query)

In [7]:
import os

class Item:
    def __init__(self, item=None, *args, **kwargs):
        self.item = item
        self.seleted = False
        
    def get(self):
        return self.item
    
       
    def type2sym(self, t):
        tmp = ''
        if t[0]: tmp += 'D'
        if t[1]: tmp += 'F'
        if t[2]: tmp += 'L'
        if t[3]: tmp += 'M'
        return tmp
        
        
    def gettype(self, convert=None):
        type = (os.path.isdir(self.item), os.path.isfile(self.item), os.path.islink(self.item), os.path.ismount(self.item))
        if convert == 'numeric':
            return ''.join(['1' if (i == True) else '0' for i in type])
        if convert == 'symbolic':
            return self.type2sym(type)
        return type
            
        return type
    
    def gettime(self, v=None, t=None):
        '''
        t - time view
        v - view format
        '''
        
        from time import gmtime
        from time import localtime
        from time import asctime
        time = (os.path.getctime(self.item), os.path.getmtime(self.item), os.path.getmtime(self.item))
        if v == 'no':
            time = ''
        if t == 'UTC':
            return ' '.join((asctime(gmtime(i)) for i in time))
        if t == 'local':
            return ' '.join((asctime(localtime(i)) for i in time))
        return time
    
    def getsize(self, path=None):
        #TODO: its dont work right
        path = path or self.item
        size = 0
        try:
            if 'D' in self.gettype(convert='symbolic'):
                size = self.getdirsize()
            else:
                size = os.path.getsize(path)
        except:
            return '-'        
        return size
    
        
    def getdirsize(self):
        #TODO: its dont work right too
        tmp = 0
        path = os.path.normpath(self.get()).replace('\\', '\\\\')
        for i in os.listdir(path):
            tmp += os.path.getsize(os.path.normpath(path + '/' + i))
        return tmp
    
    def sizeconverter(self, size, converted=True, delta=0, system='nosi'):
        #TODO: errors to log
        try:
            size = int(size)
        except:
            return '-'
        if not converted :
            return '{} b'.format(size)
        postfix = ('', 'K', 'M', 'G', 'T', 'P', 'E')
        index = 0        
        string = '{:.2f} {}b'        
        divisor = 1024 if system == 'nosi' else 1000
        while size > (divisor + delta):
            size /= divisor
            index += 1
        return string.format(size, postfix[index])
    
    def nameconverter(self, name, showpath=True):
        string = name
        if not showpath:
            return os.path.basename(string)
        return string
    

    def indexconverter(self, index, showindex=True, start=1):
        try:
            index = int(index)
        except:
            return '#'
        if not showindex:
            return ''
        return index + start
            
        
        
class DataBase(Item):
    def __init__(self, structure=None, *args, **kwargs):
        self.structure = list(structure or [])
        self.dbdata = ['{{{item}}}'.format(item=item) for item in self.structure]
        self.dbformat = '\t'.join(self.dbdata)
        self.dbprintformat = ' '.join(self.dbdata)
        
    
    

class Finder(Item):
    def __init__(self, path=None, items=None, *args, **kwargs ):
        self.path = path
        self.items = list(items or [])
        self.dbstructure = ('index','type','item','size','time')
#         self.dboutmask = (1, 1, 1, 1, 0)
        self.db = DataBase(structure=self.dbstructure)
        self.scandir(self.path)


        
    #def select(self, query):
    
    def dbprintformatmodifier(self, showindex=True, showtype=True, showitem=True, showsize=True, showtime=False):
        #TODO: WTF?!
        printformat = list(self.db.dbdata)
        if not showindex:
            printformat.remove('{index}')
        if not showtype:
            printformat.remove('{type}')
        if not showitem:
            printformat.remove('{item}')
        if not showsize:
            printformat.remove('{size}')
        if not showtime:
            printformat.remove('{time}')
        return ' '.join(printformat)
        
        
    
    def getpath(self):
        return self.path
    
    def getitems(self):
        return self.items
    
    def getsize(self):
        return len(self.items)
    
    def gettable(self):
        table = list()
        for i, item in enumerate(self.items):
            table.append((self.db.dbformat.format(index = i, 
                                              type = item.gettype(convert='symbolic'),
                                              item = item.get(), 
                                              time = item.gettime(v='no'), 
                                              size = item.getsize()
                                                                      )
                         ))
        return table
    
    def printtable(self, table):  
        for i in table:
            row = i.split('\t')
            #TODO: WTF?!
            print(self.dbprintformatmodifier(showindex=True).format(index = self.indexconverter(row[0]),
                                       type = row[1],
                                       item = self.nameconverter(row[2], showpath=False),
                                       size = self.sizeconverter(row[3]),
                                       time = row[4],
                                                               )
                 )
                  
            
    
    def scandir(self, path=None, depth=0):
        path = path or self.path
        for item in os.listdir(path):
            self.items.append(Item(item=os.path.normpath((os.path.abspath(path)+'/'+item))))
    
path = r'E:\\'
# f = Finder(path='c:\\')
f1 = Finder(path)
# print(f.getpath())
# print(f.scandir())
# print(f.getitems())
# print('-'*10)

# print(f1.getpath())/
# print(f1.scandir())

# print(Item(item=os.path.normpath(f1.getitems()[6].get()).replace('\\', '\\\\')).getdirsize())

# print(f1.getsize())
f1.printtable(f1.gettable())
# print(f1.db.dbdata)
# print(f1.db.dbformat)
# print(f1.db.dbprintformat)

# struct = ('index', 'data', 'name')
# db = DataBase(structure=struct)
# print(db.createdata())

PermissionError: [WinError 5] Отказано в доступе: 'E:\\System Volume Information'